In [1]:
import os
import sys
import numpy as np
import yaml
import argparse
import shutil
from copy import deepcopy
from os.path import join as pjoin

BASEPATH = os.path.dirname('/home/taehyun/workspace/personal_git/deep-motion-editing')
sys.path.insert(0, '/home/taehyun/workspace/personal_git/deep-motion-editing')

sys.path.insert(0, pjoin(BASEPATH))
# # sys.path.insert(0, pjoin(BASEPATH, '..', '..'))
from utils.animation_data import AnimationData
from utils.load_skeleton import Skel
from utils import BVH

In [9]:
rest = '/Users/taehyun/workspace/childtoy/MotionReasoning/style_transfer/global_info/rest.bvh'
anim_cmu, names_cmu, ft_cmu = BVH.load(rest,order='zyx')
names_cmu

['Hips',
 'LHipJoint',
 'LeftUpLeg',
 'LeftLeg',
 'LeftFoot',
 'LeftToeBase',
 'RHipJoint',
 'RightUpLeg',
 'RightLeg',
 'RightFoot',
 'RightToeBase',
 'LowerBack',
 'Spine',
 'Spine1',
 'Neck',
 'Neck1',
 'Head',
 'LeftShoulder',
 'LeftArm',
 'LeftForeArm',
 'LeftHand',
 'LeftFingerBase',
 'LeftHandIndex1',
 'LThumb',
 'RightShoulder',
 'RightArm',
 'RightForeArm',
 'RightHand',
 'RightFingerBase',
 'RightHandIndex1',
 'RThumb']

In [17]:
rest_emotion = '/Users/taehyun/workspace/childtoy/MotionReasoning/style_transfer/data/kinematic-dataset-of-actors-expressing-emotions-1.0.0/BVH/F02/F02A0V1.bvh'
anim_emo, names_emo, ft_emo = BVH.load(rest_emotion,order='xyz')
for a,b in zip(names_emo, names_cmu):
    print(a,b)

Hips Hips
RightUpLeg LHipJoint
RightLeg LeftUpLeg
RightFoot LeftLeg
LeftUpLeg LeftFoot
LeftLeg LeftToeBase
LeftFoot RHipJoint
Spine RightUpLeg
Spine1 RightLeg
Spine2 RightFoot
Spine3 RightToeBase
Neck LowerBack
Head Spine
RightShoulder Spine1
RightArm Neck
RightForeArm Neck1
RightHand Head
RightHandThumb1 LeftShoulder
RightHandThumb2 LeftArm
RightHandThumb3 LeftForeArm
RightInHandIndex LeftHand
RightHandIndex1 LeftFingerBase
RightHandIndex2 LeftHandIndex1
RightHandIndex3 LThumb
RightInHandMiddle RightShoulder
RightHandMiddle1 RightArm
RightHandMiddle2 RightForeArm
RightHandMiddle3 RightHand
RightInHandRing RightFingerBase
RightHandRing1 RightHandIndex1
RightHandRing2 RThumb


In [24]:
for a,b in enumerate(names_emo):
    print(a,b)

0 Hips
1 RightUpLeg
2 RightLeg
3 RightFoot
4 LeftUpLeg
5 LeftLeg
6 LeftFoot
7 Spine
8 Spine1
9 Spine2
10 Spine3
11 Neck
12 Head
13 RightShoulder
14 RightArm
15 RightForeArm
16 RightHand
17 RightHandThumb1
18 RightHandThumb2
19 RightHandThumb3
20 RightInHandIndex
21 RightHandIndex1
22 RightHandIndex2
23 RightHandIndex3
24 RightInHandMiddle
25 RightHandMiddle1
26 RightHandMiddle2
27 RightHandMiddle3
28 RightInHandRing
29 RightHandRing1
30 RightHandRing2
31 RightHandRing3
32 RightInHandPinky
33 RightHandPinky1
34 RightHandPinky2
35 RightHandPinky3
36 LeftShoulder
37 LeftArm
38 LeftForeArm
39 LeftHand
40 LeftHandThumb1
41 LeftHandThumb2
42 LeftHandThumb3
43 LeftInHandIndex
44 LeftHandIndex1
45 LeftHandIndex2
46 LeftHandIndex3
47 LeftInHandMiddle
48 LeftHandMiddle1
49 LeftHandMiddle2
50 LeftHandMiddle3
51 LeftInHandRing
52 LeftHandRing1
53 LeftHandRing2
54 LeftHandRing3
55 LeftInHandPinky
56 LeftHandPinky1
57 LeftHandPinky2
58 LeftHandPinky3


In [ ]:
1 LHipJoint
6 RHipJoint
11 LowerBack
14 Neck
17 LeftShoulder
21 LeftFingerBase
23 LThumb
24 RightShoulder
28 RightFingerBase


In [22]:
#Check Joint Indexing for CMU 
for a, b in enumerate(names_cmu):
    print(a,b)


0 Hips
1 LHipJoint
2 LeftUpLeg
3 LeftLeg
4 LeftFoot
5 LeftToeBase
6 RHipJoint
7 RightUpLeg
8 RightLeg
9 RightFoot
10 RightToeBase
11 LowerBack
12 Spine
13 Spine1
14 Neck
15 Neck1
16 Head
17 LeftShoulder
18 LeftArm
19 LeftForeArm
20 LeftHand
21 LeftFingerBase
22 LeftHandIndex1
23 LThumb
24 RightShoulder
25 RightArm
26 RightForeArm
27 RightHand
28 RightFingerBase
29 RightHandIndex1
30 RThumb


In [ ]:
dataset_config = '/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/global_info/emotion_dataset.yml'
with open(dataset_config, "r") as f:
    cfg = yaml.load(f, Loader=yaml.Loader)
style_names = cfg["style_names"]
style_name_to_idx = {name: i for i, name in enumerate(style_names)}

skel = Skel()

bvh_files = get_bvh_files(bvh_path)

train_inputs = []
test_inputs = []
trainfull_inputs = []

group_size = 10  # pick the last clip from every group_size clips for test
test_window = window * 2

for i, item in enumerate(bvh_files):
    print('Processing %i of %i (%s)' % (i, len(bvh_files), item))
    filename = item.split('/')[-1]
    style, _ = filename.split('_')
    style_idx = style_name_to_idx[style]

    raw = bvh_to_motion_and_phase(item, downsample=downsample, skel=skel)  # [T, xxx]
    total_length = len(raw)
    group_length = test_window * group_size

    for st in range(0, total_length, group_length):
        ed = st + group_length
        if ed <= total_length:
            test_clips = motion_and_phase_to_dict([raw[ed - test_window: ed]], style_idx, {"style": style})
            test_inputs += test_clips
        train_clips = motion_and_phase_to_dict(divide_clip_bfa(raw[st: ed - test_window],
                                                               window=window, window_step=window_step, divide=True),
                                               style_idx, {"style": style})

        trainfull_clips = motion_and_phase_to_dict(divide_clip_bfa(raw[st: ed - test_window],
                                                                   window=test_window, window_step=test_window, divide=True),
                                                   style_idx, {"style": style})
        train_inputs += train_clips
        trainfull_inputs += trainfull_clips


In [ ]:
def get_bvh_files(directory):
    return [os.path.join(directory, f) for f in sorted(list(os.listdir(directory)))
            if os.path.isfile(os.path.join(directory, f))
            and f.endswith('.bvh') and f != 'rest.bvh']



In [ ]:
skel = Skel()
bvh_path = '/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia'
bvh_files = get_bvh_files(bvh_path)

train_inputs = []
test_inputs = []
trainfull_inputs = []
test_files = []
test_cnt = {}  # indexed by content_style

In [ ]:
def motion_and_phase_to_dict(fulls, style, meta):
    """
    fulls: a list of [T, xxx + 1] - motion and phase
    style: a *number*
    meta: a dict, e.g. {"style": "angry", "content": "walk"}
    """
    output = []
    for full in fulls:
        motion, phase = full[:, :-1], full[:, -1]
        phase_label = phase[len(phase) // 2]
        meta_copy = deepcopy(meta)
        meta_copy["phase"] = phase_label
        output.append({
            "motion": motion,
            "style": style,
            "meta": meta_copy
        })
    return output
def process_file(filename, divider, window, window_step, downsample=4, skel=None, divide=True):
    input = bvh_to_motion_and_phase(filename, downsample=downsample, skel=skel)  # [T, xxx]
    return divider(input, window=window, window_step=window_step, divide=divide)

def divide_clip_xia(input, window, window_step, divide):
    if not divide:  # return the whole clip
        t = ((input.shape[0]) // 4) * 4 + 4
        t = max(t, 12)
        if len(input) < t:
            input = pad_to_window(input, t)
        return [input]

    windows = []
    j = -(window // 4)
    total = len(input)
    while True:
        slice = input[max(j, 0): j + window].copy()  # remember to COPY!!
        if len(slice) < window:
            slice = pad_to_window(slice, window)
        windows.append(slice)
        j += window_step
        if total - j < (3 * window) // 4:
            break
    return windows
def pad_to_window(slice, window):
    def get_reflection(src, tlen):  # return [src-reversed][src][src-r]...
        x = src.copy()
        x = np.flip(x, axis=0)
        ret = x.copy()
        while len(ret) < tlen:
            x = np.flip(x, axis=0)
            ret = np.concatenate((ret, x), axis=0)
        ret = ret[:tlen]
        return ret

    if len(slice) >= window:
        return slice
    left_len = (window - len(slice)) // 2 + (window - len(slice)) % 2
    right_len = (window - len(slice)) // 2
    left = np.flip(get_reflection(np.flip(slice, axis=0), left_len), axis=0)
    right = get_reflection(slice, right_len)
    slice = np.concatenate([left, slice, right], axis=0)
    assert len(slice) == window
    return slice

def bvh_to_motion_and_phase(filename, downsample, skel):
    anim = AnimationData.from_BVH(filename, downsample=downsample, skel=skel)
    full = anim.get_full()  # [T, xxx]
    print(full.shape)
    phases = anim.get_phases()  # [T, 1]
    return np.concatenate((full, phases), axis=-1)

def set_init(dic, key, value):
    try:
        dic[key]
    except KeyError:
        dic[key] = value



In [ ]:
window = 32
window_step=8
for i, item in enumerate(bvh_files):
    print('Processing %i of %i (%s)' % (i, len(bvh_files), item))
    filename = item.split('/')[-1]
    style, content_idx, _ = filename.split('_')
    content = content_namedict[int(content_idx) - 1]
    content_style = "%s_%s" % (content, style)
    
    uclip = motion_and_phase_to_dict(process_file(item, divider=divide_clip_xia, window=window, window_step=window_step,
                                                  skel=skel, divide=False),
                                     style_name_to_idx[style],
                                     {"style": style, "content": content})
    # Whether this should be a test clip
    set_init(test_cnt, content_style, 0)
    if test_cnt[content_style] < content_test_cnt[content]:
        test_cnt[content_style] += 1
        test_inputs += uclip
        test_files.append(filename)
    else:
        trainfull_inputs += uclip
        clips = motion_and_phase_to_dict(process_file(item, divider=divide_clip_xia, window=window, window_step=window_step,
                                                      skel=skel, divide=True),
                                         style_name_to_idx[style],
                                         {"style": style, "content": content})
        train_inputs += clips


In [ ]:
trainfull_inputs[102]['motion'].shape

In [ ]:
data_dict = {}
data_info = {}
for subset, inputs in zip(["train", "test", "trainfull"], [train_inputs, test_inputs, trainfull_inputs]):
    motions = [input["motion"] for input in inputs]
    styles = [input["style"] for input in inputs]
    meta = {key: [input["meta"][key] for input in inputs] for key in inputs[0]["meta"].keys()}
    data_dict[subset] = {"motion": motions, "style": styles, "meta": meta}

    """compute meta info"""
    num_clips = len(motions)
    info = {"num_clips": num_clips,
            "distribution":
                {style:
                     {content: len([i for i in range(num_clips) if meta["style"][i] == style and meta["content"][i] == content])
                      for content in content_names}
                 for style in style_names}
            }
    data_info[subset] = info


In [ ]:
motions[0].shape

In [ ]:
for subset, inputs in zip(["train", "test", "trainfull"], [train_inputs, test_inputs, trainfull_inputs]):
    print(inputs)


In [ ]:
data_dict['train']['motion'][0].shape

In [ ]:
data_dict['trainfull']

In [26]:
np.array( [[ 0.,93.594,  0.   ],
    [-9.5,-1.594,  0.   ],
    [  0.,-42.,   0.],
    [  0., -42.,   0.],
    [ 9.5,   -1.594 , 0.   ],
    [  0., -42. ,  0.],
    [  0., -42. ,  0.],
    [ 0. ,   14.348,  0.   ],
    [0.  ,  9.746, 0.   ],
    [ 0. ,   10.149,  0.   ],
    [0.  ,  9.746 ,0.   ],
    [ 0. ,   10.417,  0.   ],
    [0. ,9., 0.],
    [-3.2,    6.945,  0.   ],
    [-12.8,   0.,    0. ],
    [-27. ,  0. ,  0.],
    [-26. ,  0. ,  0.],
    [-2.56,   0.196,  3.21 ],
    [-3.788,  0.   ,  0.   ],
    [-2.631,  0.   ,  0.   ],
    [-3.316,  0.523,  2.035],
    [-5.366, -0.094,  1.028],
    [-3.723,  0.   ,  0.   ],
    [-2.111,  0.   ,  0.   ],
    [-3.479,  0.532,  0.778],
    [-5.322, -0.086,  0.323],
    [-4.062,  0.   ,  0.   ],
    [-2.547,  0.   ,  0.   ],
    [-3.461,  0.553, -0.133],
    [-4.767, -0.023, -0.493],
    [-3.541,  0.   ,  0.   ],
    [-2.456,  0.   ,  0.   ],
    [-3.251,  0.483, -1.236],
    [-4.259, -0.023, -1.122],
    [-2.835,  0.   ,  0.   ],
    [-1.792,  0.   ,  0.   ],
    [3.2   ,6.945, 0.   ],
    [12.8  ,0. ,  0. ],
    [27.  ,0.,  0.],
    [26.  ,0.,  0.],
    [2.56 , 0.196, 3.21 ],
    [3.788, 0. ,   0.   ],
    [2.631, 0. ,   0.   ],
    [3.316, 0.523, 2.035],
    [ 5.366, -0.094,  1.028],
    [3.723, 0. ,   0.   ],
    [2.111, 0. ,   0.   ],
    [3.479, 0.532, 0.778],
    [ 5.322, -0.086 , 0.323],
    [4.062, 0.  ,  0.   ],
    [2.547, 0.  ,  0.   ],
    [ 3.461,  0.553, -0.133],
    [ 4.767, -0.023, -0.493],
    [3.541, 0.  ,  0.   ],
    [2.456, 0.  ,  0.   ],
    [ 3.251,  0.483, -1.236],
    [ 4.259, -0.023, -1.122],
    [2.835, 0. ,   0.   ],
[1.792, 0.,    0.   ]]).shape

(59, 3)

In [27]:
len(    [-1,  0,  1,  2,  0,  4,  5,  0,  7,  8,  9, 10, 11, 10, 13, 14, 15,
       16, 17, 18, 16, 20, 21, 22, 16, 24, 25, 26, 16, 28, 29, 30, 16, 32,
       33, 34, 10, 36, 37, 38, 39, 40, 41, 39, 43, 44, 45, 39, 47, 48, 49,
       39, 51, 52, 53, 39, 55, 56, 57])

59